## นพกร แผ่นผา 6010405335

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import VarianceThreshold

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel


In [6]:
def encoder(dataset):
    temp = dataset.copy()
    #create mapping
    labels = temp['workclass'].astype('category').cat.categories.tolist()
    replace_map_workclass = {'workclass': {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['education'].astype('category').cat.categories.tolist()
    replace_map_education = {'education' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['marital status'].astype('category').cat.categories.tolist()
    replace_map_marital = {'marital status' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['occupation'].astype('category').cat.categories.tolist()
    replace_map_occupation = {'occupation' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['relationship'].astype('category').cat.categories.tolist()
    replace_map_relationship = {'relationship' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['race'].astype('category').cat.categories.tolist()
    replace_map_race = {'race' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['sex'].astype('category').cat.categories.tolist()
    replace_map_sex = {'sex' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['native country'].astype('category').cat.categories.tolist()
    replace_map_nativecountry = {'native country' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}

    labels = temp['label'].astype('category').cat.categories.tolist()
    replace_map_label = {'label' : {k: v for k, v in zip(labels, list(range(1, len(labels) + 1)))}}
    #replace map to categorycal
    temp.replace(replace_map_workclass, inplace=True)
    temp.replace(replace_map_education, inplace=True)
    temp.replace(replace_map_marital, inplace=True)
    temp.replace(replace_map_occupation,inplace=True)
    temp.replace(replace_map_relationship,inplace=True)
    temp.replace(replace_map_race,inplace=True)
    temp.replace(replace_map_sex,inplace=True)
    temp.replace(replace_map_nativecountry,inplace=True)
    temp.replace(replace_map_label,inplace=True)
    #print(temp.iloc[:,5:9].head(5))
    return temp,replace_map_label

In [7]:
dataset = pd.read_csv("census-income-train.csv")
dataset

,age,workclass,weight,education,edu num,marital status,occupation,relationship,race,sex,captial-gain,capital-loss,hours-per-week,native country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,No
1,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,No
2,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,Yes
3,30,State-gov,141297,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,Yes
4,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,?,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33869,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,No
33870,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,No
33871,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,No
33872,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,No


In [8]:
dataset.loc[dataset.workclass == " ?"]

,age,workclass,weight,education,edu num,marital status,occupation,relationship,race,sex,captial-gain,capital-loss,hours-per-week,native country,label
37,32,?,293936,7th-8th,4,Married-spouse-absent,?,Not-in-family,White,Male,0,0,40,?,No
45,67,?,212759,10th,6,Married-civ-spouse,?,Husband,White,Male,0,0,2,United-States,No
73,35,?,129305,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,40,United-States,No
87,43,?,174662,Some-college,10,Divorced,?,Not-in-family,White,Female,0,0,40,United-States,No
97,68,?,38317,1st-4th,2,Divorced,?,Not-in-family,White,Female,0,0,20,United-States,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33719,56,?,124319,Masters,14,Married-civ-spouse,?,Husband,White,Male,0,0,40,United-States,Yes
33777,61,?,265201,Some-college,10,Married-civ-spouse,?,Husband,White,Male,0,0,14,United-States,No
33829,21,?,212661,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,No
33854,81,?,26711,Assoc-voc,11,Married-civ-spouse,?,Husband,White,Male,2936,0,20,United-States,No


In [9]:
dataset = dataset.replace(to_replace=" ?", value=np.nan)

In [10]:
dataset.isna().sum()

age                  0
workclass         1979
weight               0
education            0
edu num              0
marital status       0
occupation        1984
relationship         0
race                 0
sex                  0
captial-gain         0
capital-loss         0
hours-per-week       0
native country     593
label                0
dtype: int64

In [11]:
dataset

,age,workclass,weight,education,edu num,marital status,occupation,relationship,race,sex,captial-gain,capital-loss,hours-per-week,native country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,No
1,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,No
2,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,Yes
3,30,State-gov,141297,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,India,Yes
4,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,NaN,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33869,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,No
33870,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,No
33871,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,No
33872,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,No


In [12]:
dataset,lables = encoder(dataset)

In [13]:
dataset.isna().sum()

age                  0
workclass         1979
weight               0
education            0
edu num              0
marital status       0
occupation        1984
relationship         0
race                 0
sex                  0
captial-gain         0
capital-loss         0
hours-per-week       0
native country     593
label                0
dtype: int64

In [14]:
dataset.workclass.replace(np.nan,dataset.workclass.mean(),inplace=True)
dataset.occupation.replace(np.nan,dataset.occupation.mean(),inplace=True)
dataset["native country"].replace(np.nan,dataset["native country"].mean(),inplace=True)


In [15]:
dataset.isna().sum()

age               0
workclass         0
weight            0
education         0
edu num           0
marital status    0
occupation        0
relationship      0
race              0
sex               0
captial-gain      0
capital-loss      0
hours-per-week    0
native country    0
label             0
dtype: int64

In [16]:
X = dataset.values[:,0:14]
Y = dataset.values[:,14]

In [17]:
scalar = StandardScaler().fit(X)
X = scalar.transform(X)

In [18]:
linearSvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, Y)
model = SelectFromModel(linearSvc, prefit=True)
X = model.transform(X)

In [19]:
x_train,x_validate,y_train,y_validate = train_test_split(X,Y,test_size=0.2,random_state=1)

In [20]:
sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train.ravel())

In [21]:
#model = KNeighborsClassifier(n_neighbors=3,algorithm="kd_tree")

In [22]:
model = SVC(kernel = 'rbf', gamma ="auto",C = 1.0,)

In [23]:
model.fit(x_train_res, y_train_res)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [24]:
prediction = model.predict(x_validate)

In [25]:
print("No :",lables.get("label").get("No"))
print("Yes :",lables.get("label").get("Yes"))
#print(dataset.label.value_counts())
print("accuracy score",accuracy_score(y_validate,prediction))
print("confusion matrix")
print("",confusion_matrix(y_validate,prediction))
print(classification_report(y_validate, prediction))
print("geometric mean score",geometric_mean_score(y_validate,prediction))

No : 1
Yes : 2
accuracy score 0.7976383763837639
confusion matrix
 [[3981 1152]
 [ 219 1423]]
              precision    recall  f1-score   support

         1.0       0.95      0.78      0.85      5133
         2.0       0.55      0.87      0.67      1642

    accuracy                           0.80      6775
   macro avg       0.75      0.82      0.76      6775
weighted avg       0.85      0.80      0.81      6775

geometric mean score 0.8198347644959878
